In [1]:
project_id = 'elife-data-pipeline'
source_dataset = 'de_dev'
output_dataset = 'de_dev'
output_table_prefix = 'data_science_'
output_table_name = 'Forecast_Initial_Submission'
sql_filename = 'initial_submission_count_by_date.sql'
# output_table_name = 'Forecast_Full_Submission_Received_To_First_Decision'
# sql_filename = 'full_submission_received_to_first_decision_by_date.sql'
# output_table_name = 'Forecast_Accept_Decision_To_VOR'
# sql_filename = 'accept_decision_to_vor_by_date.sql'
ewma_span = 30
rolling_average_window = 30

In [2]:
from functools import partial

import pandas as pd

import data_science_pipeline.configure_warnings  # pylint: disable=unused-import, wrong-import-order

from fbprophet import Prophet

from IPython.display import display

from data_science_pipeline.sql import get_sql
from data_science_pipeline.utils.bq import to_gbq
from data_science_pipeline.utils.jupyter import (
    read_big_query as _read_big_query,
)
from data_science_pipeline.utils.timeseries import (
    to_date_isoformat,
    to_timestamp_isoformat,
    get_month_start_date,
    get_quarter_start_date,
    get_year_start_date,
    get_quarter_week_date,
    filter_by_month,
    filter_by_quarter,
    filter_by_year,
    get_rolling_average,
    get_ewma
)

import data_science_pipeline.configure_notebook_logging  # pylint: disable=unused-import, ungrouped-imports

In [3]:
forecast_output_table_name = '{output_dataset}.{prefix}{suffix}'.format(
    output_dataset=output_dataset,
    prefix=output_table_prefix,
    suffix=output_table_name
)

In [4]:
read_big_query = partial(_read_big_query, project_id=project_id)

In [5]:
default_query_props = dict(project=project_id, dataset=source_dataset)

In [6]:
raw_timeseries_df = read_big_query(
    get_sql(sql_filename).format(
        **default_query_props
    )
)
raw_timeseries_df.head()

> ```sql
> SELECT
>   DATE(QC_Complete_Timestamp) AS initial_submission_date,
>   COUNT(DISTINCT Manuscript_ID) AS manuscript_count
> FROM `elife-data-pipeline.de_dev.mv_Editorial_Manuscript_Version`
> WHERE Overall_Stage = 'Initial Submission'
>   AND Position_In_Overall_Stage = 1
>   AND DATE(QC_Complete_Timestamp) < DATE(CURRENT_TIMESTAMP())
> GROUP BY initial_submission_date
> ORDER BY initial_submission_date DESC
> ```

,initial_submission_date,manuscript_count
0,2020-10-01,24
1,2020-09-30,36
2,2020-09-29,32
3,2020-09-28,36
4,2020-09-27,16


In [7]:
raw_timeseries_df.dtypes

initial_submission_date    datetime64[ns]
manuscript_count                    int64
dtype: object

In [8]:
assert len(raw_timeseries_df.columns) == 2

In [9]:
assert pd.api.types.is_datetime64_dtype(raw_timeseries_df.dtypes[0])

In [10]:
assert pd.api.types.is_integer_dtype(raw_timeseries_df.dtypes[1])

In [11]:
timeseries_df = raw_timeseries_df.copy()
timeseries_df.columns = ['ds', 'y']
timeseries_df.head()

,ds,y
0,2020-10-01,24
1,2020-09-30,36
2,2020-09-29,32
3,2020-09-28,36
4,2020-09-27,16


In [12]:
model = Prophet(daily_seasonality=True)
model.fit(timeseries_df)
None

In [13]:
forecast_df = model.predict(
    model.make_future_dataframe(periods=365 + 31, include_history=False)
)

In [14]:
forecast_df[['ds', 'yhat_lower', 'yhat', 'yhat_upper']].head(30)

,ds,yhat_lower,yhat,yhat_upper
0,2020-10-02,23.466968,32.503172,41.409815
1,2020-10-03,12.043781,21.499029,30.596949
2,2020-10-04,10.806049,20.024258,29.199456
3,2020-10-05,20.546451,30.075427,39.203784
4,2020-10-06,24.862623,33.703488,42.386024
5,2020-10-07,24.768181,33.887211,43.281204
6,2020-10-08,24.045014,33.368907,42.135448
7,2020-10-09,23.048996,32.354638,41.635272
8,2020-10-10,12.112745,21.316022,30.050477
9,2020-10-11,11.273221,19.812992,28.983538


In [15]:
prediction_start_date = forecast_df['ds'][0] + pd.offsets.DateOffset(days=0)
print('prediction_start_date:', prediction_start_date)
prediction_month_start_date = get_month_start_date(prediction_start_date)
print('prediction_month_start_date:', prediction_month_start_date)
prediction_quarter_start_date = get_quarter_start_date(prediction_start_date)
print('prediction_quarter_start_date:', prediction_quarter_start_date)
prediction_year_start_date = get_year_start_date(prediction_start_date)
print('prediction_year_start_date:', prediction_year_start_date)

prediction_start_date: 2020-10-02 00:00:00
prediction_month_start_date: 2020-10-01 00:00:00
prediction_quarter_start_date: 2020-10-01 00:00:00
prediction_year_start_date: 2020-01-01 00:00:00


In [16]:
_df = filter_by_month(forecast_df, prediction_month_start_date)
print(len(_df))
display(_df['ds'].head())
display(_df['ds'].tail())

30


0   2020-10-02
1   2020-10-03
2   2020-10-04
3   2020-10-05
4   2020-10-06
Name: ds, dtype: datetime64[ns]

25   2020-10-27
26   2020-10-28
27   2020-10-29
28   2020-10-30
29   2020-10-31
Name: ds, dtype: datetime64[ns]

In [17]:
_df = filter_by_quarter(forecast_df, prediction_quarter_start_date)
print(len(_df))
display(_df['ds'].head())
display(_df['ds'].tail())

91


0   2020-10-02
1   2020-10-03
2   2020-10-04
3   2020-10-05
4   2020-10-06
Name: ds, dtype: datetime64[ns]

86   2020-12-27
87   2020-12-28
88   2020-12-29
89   2020-12-30
90   2020-12-31
Name: ds, dtype: datetime64[ns]

In [18]:
_df = filter_by_year(forecast_df, prediction_year_start_date)
print(len(_df))
display(_df['ds'].head())
display(_df['ds'].tail())

91


0   2020-10-02
1   2020-10-03
2   2020-10-04
3   2020-10-05
4   2020-10-06
Name: ds, dtype: datetime64[ns]

86   2020-12-27
87   2020-12-28
88   2020-12-29
89   2020-12-30
90   2020-12-31
Name: ds, dtype: datetime64[ns]

In [19]:
_month_dates = [
    prediction_month_start_date + pd.offsets.MonthBegin(month_offset)
    for month_offset in range(-12, 13)
]
predicted_month_count_incl_history_df = pd.DataFrame([
    {
        'ds': month_date,
        'y_actual': filter_by_month(timeseries_df, month_date)['y'].sum(),
        'y_forecast': filter_by_month(forecast_df, month_date)['yhat'].sum()
    }
    for month_date in _month_dates
])
predicted_month_count_incl_history_df['y'] = (
    predicted_month_count_incl_history_df['y_actual']
    + predicted_month_count_incl_history_df['y_forecast']
)
predicted_month_count_incl_history_df.head(25)

,ds,y_actual,y_forecast,y
0,2019-10-01,825,0.000000,825.000000
1,2019-11-01,768,0.000000,768.000000
2,2019-12-01,684,0.000000,684.000000
3,2020-01-01,758,0.000000,758.000000
4,2020-02-01,759,0.000000,759.000000
5,2020-03-01,843,0.000000,843.000000
6,2020-04-01,891,0.000000,891.000000
7,2020-05-01,951,0.000000,951.000000
8,2020-06-01,942,0.000000,942.000000
9,2020-07-01,958,0.000000,958.000000


In [20]:
def add_initial(ser, initial_value):
    _values = ser.values.copy()
    _values[0] += initial_value
    return _values


forecast_result_df = (
    forecast_df[['ds', 'yhat_lower', 'yhat', 'yhat_upper']]
    .rename(columns={
        'ds': 'Forecast_Date',
        'yhat': 'Forecast_Value',
        'yhat_lower': 'Forecast_Lower_Bound_Value',
        'yhat_upper': 'Forecast_Upper_Bound_Value'
    })
    .copy()
)

for c in ['Forecast_Value', 'Forecast_Lower_Bound_Value', 'Forecast_Upper_Bound_Value']:
    forecast_result_df[c] = forecast_result_df[c].round().astype(int)

forecast_result_df['Forecast_Rolling_Average_Value'] = get_rolling_average(
    forecast_result_df['Forecast_Value'],
    window=rolling_average_window
)

forecast_result_df['Forecast_Ewma_Value'] = get_ewma(
    forecast_result_df['Forecast_Value'],
    span=ewma_span
)

forecast_result_df['Forecast_Month_Offset_Value'] = add_initial(
    forecast_result_df['Forecast_Value'],
    filter_by_month(timeseries_df, prediction_month_start_date)['y'].sum()
)
forecast_result_df['Forecast_Month_Offset_Value'] = add_initial(
    forecast_result_df['Forecast_Value'],
    filter_by_quarter(timeseries_df, prediction_month_start_date)['y'].sum()
)
forecast_result_df['Forecast_Quarter_Offset_Value'] = add_initial(
    forecast_result_df['Forecast_Value'],
    filter_by_quarter(timeseries_df, prediction_quarter_start_date)['y'].sum()
)
forecast_result_df['Forecast_Year_Offset_Value'] = add_initial(
    forecast_result_df['Forecast_Value'],
    filter_by_year(timeseries_df, prediction_year_start_date)['y'].sum()
)
forecast_result_df['Forecast_Date'] = forecast_result_df['Forecast_Date'].apply(to_date_isoformat)
forecast_result_df['Date'] = forecast_result_df['Forecast_Date']
forecast_result_df['Model'] = 'fbprophet'
print(len(forecast_result_df))
forecast_result_df.head()

396


,Forecast_Date,Forecast_Lower_Bound_Value,Forecast_Value,Forecast_Upper_Bound_Value,Forecast_Rolling_Average_Value,Forecast_Ewma_Value,Forecast_Month_Offset_Value,Forecast_Quarter_Offset_Value,Forecast_Year_Offset_Value,Date,Model
0,2020-10-02,23,33,41,29.333333,33.000000,57,57,7878,2020-10-02,fbprophet
1,2020-10-03,12,21,31,28.812500,26.800000,21,21,21,2020-10-03,fbprophet
2,2020-10-04,11,20,29,28.294118,24.380600,20,20,20,2020-10-04,fbprophet
3,2020-10-05,21,30,39,28.388889,25.928949,30,30,30,2020-10-05,fbprophet
4,2020-10-06,25,34,42,28.631579,27.765309,34,34,34,2020-10-06,fbprophet


In [21]:
actual_result_df = timeseries_df.rename(columns={'ds': 'Date', 'y': 'Actual_Value'}).copy()
actual_result_df['Date'] = actual_result_df['Date'].apply(to_date_isoformat)

actual_result_df['Actual_Rolling_Average_Value'] = get_rolling_average(
    actual_result_df['Actual_Value'],
    window=rolling_average_window
)

actual_result_df['Actual_Ewma_Value'] = get_ewma(
    actual_result_df['Actual_Value'],
    span=ewma_span
)
actual_result_df.head()

,Date,Actual_Value,Actual_Rolling_Average_Value,Actual_Ewma_Value
0,2020-10-01,24,28.133333,24.000000
1,2020-09-30,36,28.875000,30.200000
2,2020-09-29,32,29.529412,30.840429
3,2020-09-28,36,29.166667,32.262079
4,2020-09-27,16,28.157895,28.562062


In [22]:
forecast_result_with_actual_df = forecast_result_df.merge(
    actual_result_df,
    on='Date',
    how='outer'
).sort_values('Date')

forecast_result_with_actual_df['Actual_Or_Forecast_Value'] = (
    forecast_result_with_actual_df['Actual_Value'].combine_first(
        forecast_result_with_actual_df['Forecast_Value']
    )
)

forecast_result_with_actual_df['Date_Timestamp'] = (
    pd.to_datetime(forecast_result_with_actual_df['Date'])
    .apply(to_timestamp_isoformat)
)
forecast_result_with_actual_df['Month_Start_Date'] = (
    pd.to_datetime(forecast_result_with_actual_df['Date']).apply(get_month_start_date)
    .apply(to_date_isoformat)
)
forecast_result_with_actual_df['Quarter_Start_Date'] = (
    pd.to_datetime(forecast_result_with_actual_df['Date']).apply(get_quarter_start_date)
    .apply(to_date_isoformat)
)
forecast_result_with_actual_df['Year_Start_Date'] = (
    pd.to_datetime(forecast_result_with_actual_df['Date']).apply(get_year_start_date)
    .apply(to_date_isoformat)
)
forecast_result_with_actual_df['Quarter_Week_Date'] = (
    pd.to_datetime(forecast_result_with_actual_df['Date']).apply(get_quarter_week_date)
    .apply(to_date_isoformat)
)
forecast_result_with_actual_df['Is_Current_Quarter'] = (
    pd.to_datetime(forecast_result_with_actual_df['Date']).apply(get_quarter_start_date)
    == prediction_quarter_start_date
)
print(len(forecast_result_with_actual_df))
forecast_result_with_actual_df.sort_values('Date').head()

3211


,Forecast_Date,Forecast_Lower_Bound_Value,Forecast_Value,Forecast_Upper_Bound_Value,Forecast_Rolling_Average_Value,Forecast_Ewma_Value,Forecast_Month_Offset_Value,Forecast_Quarter_Offset_Value,Forecast_Year_Offset_Value,Date,...,Actual_Value,Actual_Rolling_Average_Value,Actual_Ewma_Value,Actual_Or_Forecast_Value,Date_Timestamp,Month_Start_Date,Quarter_Start_Date,Year_Start_Date,Quarter_Week_Date,Is_Current_Quarter
3210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-05-04,...,1.0,1.625000,1.746441,1.0,2012-05-04T00:00:00Z,2012-05-01,2012-04-01,2012-01-01,2012-04-30,False
3209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-05-07,...,1.0,1.647059,1.797919,1.0,2012-05-07T00:00:00Z,2012-05-01,2012-04-01,2012-01-01,2012-05-07,False
3208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-05-19,...,1.0,1.722222,1.852948,1.0,2012-05-19T00:00:00Z,2012-05-01,2012-04-01,2012-01-01,2012-05-14,False
3207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-05-22,...,1.0,1.684211,1.911772,1.0,2012-05-22T00:00:00Z,2012-05-01,2012-04-01,2012-01-01,2012-05-21,False
3206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-05-30,...,1.0,1.700000,1.974653,1.0,2012-05-30T00:00:00Z,2012-05-01,2012-04-01,2012-01-01,2012-05-28,False


In [23]:
forecast_result_with_actual_df['Actual_Month_Cumsum_Value'] = (
    forecast_result_with_actual_df.groupby('Month_Start_Date')['Actual_Value'].cumsum()
)
forecast_result_with_actual_df['Actual_Quarter_Cumsum_Value'] = (
    forecast_result_with_actual_df.groupby('Quarter_Start_Date')['Actual_Value'].cumsum()
)
forecast_result_with_actual_df['Actual_Year_Cumsum_Value'] = (
    forecast_result_with_actual_df.groupby('Year_Start_Date')['Actual_Value'].cumsum()
)
forecast_result_with_actual_df.head()

,Forecast_Date,Forecast_Lower_Bound_Value,Forecast_Value,Forecast_Upper_Bound_Value,Forecast_Rolling_Average_Value,Forecast_Ewma_Value,Forecast_Month_Offset_Value,Forecast_Quarter_Offset_Value,Forecast_Year_Offset_Value,Date,...,Actual_Or_Forecast_Value,Date_Timestamp,Month_Start_Date,Quarter_Start_Date,Year_Start_Date,Quarter_Week_Date,Is_Current_Quarter,Actual_Month_Cumsum_Value,Actual_Quarter_Cumsum_Value,Actual_Year_Cumsum_Value
3210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-05-04,...,1.0,2012-05-04T00:00:00Z,2012-05-01,2012-04-01,2012-01-01,2012-04-30,False,1.0,1.0,1.0
3209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-05-07,...,1.0,2012-05-07T00:00:00Z,2012-05-01,2012-04-01,2012-01-01,2012-05-07,False,2.0,2.0,2.0
3208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-05-19,...,1.0,2012-05-19T00:00:00Z,2012-05-01,2012-04-01,2012-01-01,2012-05-14,False,3.0,3.0,3.0
3207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-05-22,...,1.0,2012-05-22T00:00:00Z,2012-05-01,2012-04-01,2012-01-01,2012-05-21,False,4.0,4.0,4.0
3206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-05-30,...,1.0,2012-05-30T00:00:00Z,2012-05-01,2012-04-01,2012-01-01,2012-05-28,False,5.0,5.0,5.0


In [24]:
print('writing to:', forecast_output_table_name)
to_gbq(
    forecast_result_with_actual_df,
    forecast_output_table_name,
    project_id=project_id,
    if_exists='replace'
)
print('done')

writing to: de_dev.data_science_Forecast_Initial_Submission


INFO:root:Processing line 1000
INFO:root:Processing line 2000
INFO:root:Processing line 3000
INFO:root:Processed 3211 lines
INFO:data_science_pipeline.utils.bq:loading from /tmp/tmpfj5v628c/data.jsonl.gz
INFO:data_science_pipeline.utils.bq:Loaded 3211 rows into de_dev:data_science_Forecast_Initial_Submission.


done
